In [ ]:
# Eric's modification
import os
import torch
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
from torch.utils.data import TensorDataset, DataLoader, Subset
import torchvision
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
from datetime import datetime
print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
from torch.utils.data import TensorDataset, DataLoader
from os import path

from src.utils import *
from src.models import *
from src.methods import *

data_path = os.path.join(os.getcwd(), 'project_data/')
print(data_path)

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

In [ ]:
X_test = np.load(data_path+"X_test.npy")
y_test = np.load(data_path+"y_test.npy")
person_train_valid = np.load(data_path+"person_train_valid.npy")
X_train_valid = np.load(data_path+"X_train_valid.npy")
y_train_valid = np.load(data_path+"y_train_valid.npy")
person_test = np.load(data_path+"person_test.npy")
#:\Users\pluto\Desktop\2024_winter\ECE_147\final_project\ECE247-Group-Project\src\project_data\y_test.npy

In [ ]:
X_test = np.load(data_path + "X_test.npy")
y_test = np.load(data_path + "y_test.npy") - 769
person_train_valid = np.load(data_path + "person_train_valid.npy")
X_train_valid = np.load(data_path + "X_train_valid.npy")
y_train_valid = np.load(data_path + "y_train_valid.npy") - 769
person_test = np.load(data_path + "person_test.npy")

In [ ]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

In [ ]:
X_train_valid = X_train_valid[:, np.newaxis, :, :]
X_test = X_test[:, np.newaxis, :, :]

In [ ]:
X_train_valid = torch.from_numpy(X_train_valid).float()
y_train_valid = torch.from_numpy(y_train_valid).long()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

train_valid_dataset = TensorDataset(X_train_valid, y_train_valid)
test_dataset = TensorDataset(X_test, y_test)

# Splitting the dataset into train and valid sets
num_train = int(0.8 * len(train_valid_dataset))
num_valid = len(train_valid_dataset) - num_train
train_indices, valid_indices = random_split(range(len(train_valid_dataset)), [num_train, num_valid])

train_dataset = Subset(train_valid_dataset, train_indices)
valid_dataset = Subset(train_valid_dataset, valid_indices)

# Wrapping datasets with GaussianNoisyDataset
train_dataset_noisy = GaussianNoisyDataset(train_dataset, mean=0., std=1.)
valid_dataset_noisy = GaussianNoisyDataset(valid_dataset, mean=0., std=1.)
test_dataset_noisy = GaussianNoisyDataset(test_dataset, mean=0., std=1.)

batch_size = 32
train_loader = DataLoader(train_dataset_noisy, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset_noisy, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset_noisy, batch_size=batch_size, shuffle=False)

print('Training/Valid data shape:', X_train_valid.shape)
print('Test data shape:', X_test.shape)

In [ ]:
time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
writer = SummaryWriter(f'runs/ConvNet+{time}')
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-6, weight_decay=0.0005)

train(model, train_loader, valid_loader, criterion, optimizer, device=device, writer=writer, epochs=50)

test_accuracy = evaluate(model, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')
writer.add_scalar('test_accuracy',test_accuracy)
writer.close()

In [ ]:
time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
writer = SummaryWriter(f'runs/LSTM+{time}')
lstm = LSTMModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-6, weight_decay=0.0005)

lstm = lstm.to(device)
train(lstm, train_loader, valid_loader, criterion, optimizer, writer=writer, device=device)

test_accuracy = evaluate(lstm, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

In [ ]:
time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
writer = SummaryWriter(f'runs/EEG+{time}')
eegn = EEGNet_Modified()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(eegn.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

eegn = eegn.to(device)
train(eegn, train_loader, valid_loader, criterion, optimizer, device=device, epochs=100)

test_accuracy = evaluate(eegn, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

In [ ]:
time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
writer = SummaryWriter(f'runs/CNN_LSTM+{time}')
cnn_lstm = CNN_LSTM()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_lstm.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-6)

cnn_lstm = cnn_lstm.to(device)
train(cnn_lstm, train_loader, valid_loader, criterion, optimizer, writer=writer, device=device, epochs=50)

test_accuracy = evaluate(cnn_lstm, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

In [ ]:
time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
writer = SummaryWriter(f'runs/UltimateConvNet+{time}')
ult_cnn = UltimateConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ult_cnn.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-6, weight_decay=0.0005)

ult_cnn = ult_cnn.to(device)
train(ult_cnn, train_loader, valid_loader, criterion, optimizer, device=device, writer=writer, epochs=200)

test_accuracy = evaluate(ult_cnn, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

In [ ]:
# utilize ray tune to find best hyperparameters
import ray
from ray import tune
from ray import train as raytrain
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from src.methods_ray import train_ray_Ultimateconfig

config = {
    'lr': tune.loguniform(1e-4, 1e-2),
    'weight_decay': tune.loguniform(1e-4, 1e-3)
}

# Initialize Ray Tune
ray.shutdown()
ray.init()
scheduler = ASHAScheduler(metric='loss', mode='min')
reporter = CLIReporter(metric_columns=['loss'])

# Execute the hyperparameter search
analysis = tune.run(
    train_ray_Ultimateconfig,
    resources_per_trial={'gpu': 1},  # If you want to use GPUs, you can set {'gpu': 1} instead
    config=config,
    num_samples=100,
    scheduler=scheduler,
    progress_reporter=reporter
)

# Get the best hyperparameters
best_config = analysis.get_best_config(metric='loss', mode='min')
print('Best hyperparameters found are: ', best_config)

# get best accuracy using tune config


In [ ]:
# run with best hyperparameters
ult_cnn = UltimateConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ult_cnn.parameters(), lr = best_config['lr'].real, betas=(0.9, 0.99), eps=1e-6, weight_decay=best_config['weight_decay'].real)

ult_cnn = ult_cnn.to(device)
train(ult_cnn, train_loader, valid_loader, criterion, optimizer, device=device, epochs=100,patience=100)

test_accuracy = evaluate(ult_cnn, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

In [ ]:
import ray
from ray import tune
from ray import train as raytrain
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from src.methods_ray import train_ray_Ultimateconfig
from ray.tune.search.bayesopt import BayesOptSearch
ray.shutdown()
ray.init()
scheduler = ASHAScheduler(metric='loss', mode='min', max_t= 250)
reporter = CLIReporter(metric_columns=['loss'])
bayesopt = BayesOptSearch(
    metric='loss',
    mode='min',
)

# config = {
#     'weight_decay': tune.loguniform(1e-4, 1e-3),
# }

config = {
    'lr': tune.loguniform(1e-4, 1e-2),
}


analysis = tune.run(
    train_ray_Ultimateconfig,
    resources_per_trial={'gpu': 1},  # If you want to use GPUs, you can set {'gpu': 1} instead
    config=config,
    num_samples=20,
    search_alg=bayesopt,
    scheduler=scheduler,
    progress_reporter=reporter
)

# Get the best hyperparameters
best_config = analysis.get_best_config(metric='loss', mode='min')
print('Best hyperparameters found are: ', best_config)

In [ ]:
dfs = analysis.trial_dataframes
import matplotlib.pyplot as plt

for d in dfs.values():
    # TODO: Plot the results
    plt.plot(d['training_iteration'], d['loss'], label=d['config/weight_decay'][0])

plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Trial Progress Over Time')
plt.show()

In [ ]:
# run with best hyperparameters
ult_cnn = UltimateConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ult_cnn.parameters(), lr = 0.00734, betas=(0.9, 0.99), eps=1e-6, weight_decay=0.0004370)

ult_cnn = ult_cnn.to(device)
train(ult_cnn, train_loader, valid_loader, criterion, optimizer, device=device, epochs=200,patience=40)

test_accuracy = evaluate(ult_cnn, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

In [ ]:
vit = ViT(patch_size=(22, 1), num_classes=4, dim=64, num_head=8, num_layers = 2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vit.parameters(), lr=0.01, momentum=0.9)

vit = vit.to(device)
train(vit, train_loader, valid_loader, criterion, optimizer, device=device, epochs=200)

test_accuracy = evaluate(vit, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

In [ ]:
eegn = EEGNet_Modified()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(eegn.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

eegn = eegn.to(device)
train(eegn, train_loader, valid_loader, criterion, optimizer, device=device, epochs=100)

test_accuracy = evaluate(eegn, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')